In [2]:
import itertools
import json
import pickle
import re

import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
with open("../../data/pseudowords/CoMaPP_all.json") as json_file:
    data = json.load(json_file)
    
data = [{"example": d["target1"], "cue": " ".join(d["target1"].split()[:d["query_idx"]]), "pseudoword": d["label"]} for d in data]
df = pd.DataFrame.from_dict(data).drop_duplicates(ignore_index=True)
df

,example,cue,pseudoword
0,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
1,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
2,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
3,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
4,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
12128,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12129,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
12130,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99
12131,"An seiner Hand geht ein Mädchen , einen Kopf k...","An seiner Hand geht ein Mädchen , einen Kopf k...",er99


In [4]:
df['index'] = df['pseudoword'].str.extract('(\d+)').astype(int)
df.set_index('index', inplace=True)

df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [5]:
contextleft = pd.read_pickle("../../data/pseudowords/contextleft_text.pickle")

def update_cue(row):
    output = row[['example', 'cue']]
    if row['cue'] == '':  # if the string in cue is empty
        # match the index of row with contextleft['construction_id'] and match contextleft['text'] with row['example'] and create matching_entry
        matching_entry = contextleft.loc[(contextleft['construction_id'] == row.name) & (contextleft['text'] == row['example']), 'contextleft'].tolist()
        if len(matching_entry) > 0:
            output = [matching_entry[0] + " " + row['example'], matching_entry[0]]
    return output

# Add the left context if there is no cue up until the pseudoword.
df[["example", "cue"]] = df.apply(update_cue, axis=1)
df

,example,cue,pseudoword
index,,,
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,denn10
10,Und dann ist da noch das generelle Problem mit...,Und dann ist da noch das generelle Problem mit...,geschweige10
...,...,...,...
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,""" Eine low fee - Schule ist nicht automatisch ...",""" Eine low fee - Schule ist nicht automatisch ...",er99
99,"Im Gang sieht sie plötzlich Jamie , erkennt ih...","Im Gang sieht sie plötzlich Jamie , erkennt ih...",er99


In [6]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'construction'}, inplace=True)

result_df = df.groupby(['construction', 'pseudoword']).agg({'example': list, 'cue': list})

result_df

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  
construction pseudoword                                                     
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...  
             erst5       [Trainer Lucien Favre hatte schon seine beiden...  
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...  
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...  
             recht5      [Trainer Lucien Favre hatte schon seine beiden...  
...                                                                    ...  
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...  
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...  
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...  
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...  
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...  

[562 rows x 2 columns]

In [7]:
with open("../../out/definitions.pickle", "rb") as definitions_file:
    definitions = pd.DataFrame.from_dict(pickle.load(definitions_file), orient="index", columns=["definition"])
    
definitions

,definition
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ..."
...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e..."
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z..."


In [8]:
examples = pd.merge(result_df, definitions, how="inner", left_on="construction", right_index=True)
examples

example  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert und sch...   
             nicht5      [Es hat Afghanistan nicht stabilisiert und sch...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist Gold ., Schw...   
             Silber1884  [Schweigen ist Silber , reden ist Gold ., Schw...   
             ist1884     [Schweigen ist Silber , reden ist Gold ., Schw...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren war für das Ries...   
1987         sehr1987    [Solche Menschen können sich sehr wohlfühlen o...   

                                                                       cue  \
construction pseudoword                                                      
5            Und5        [""Nicht herauskaufen"", ""nicht erpressen las...   
             erst5       [Trainer Lucien Favre hatte schon seine beiden...   
             gar5        [Es hat Afghanistan nicht stabilisiert, Es hat...   
             nicht5      [Es, Es hat Afghanistan, Dass, Dass es in der ...   
             recht5      [Trainer Lucien Favre hatte schon seine beiden...   
...                                                                    ...   
1884         Gold1884    [Schweigen ist Silber , reden ist, Schweigen i...   
             Silber1884  [Schweigen ist, Schweigen ist, Angeben ist, Sc...   
             ist1884     [Schweigen, Schweigen, Angeben, Schweigen, " Q...   
1986         kaum1986    [Die Vorhut vor 20.000 Jahren, Die Vorhut vor ...   
1987         sehr1987    [Solche Menschen können, Solche Menschen könne...   

                                                                definition  
construction pseudoword                                                     
5            Und5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             erst5       Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             gar5        Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             nicht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
             recht5      Die "Negation:NEG_Xund_schon_gar_nichtY-Konstr...  
...                                                                    ...  
1884         Gold1884    Die "Intensivierung_Komparativ:Xist_SilberYist...  
             Silber1884  Die "Intensivierung_Komparativ:Xist_SilberYist...  
             ist1884     Die "Intensivierung_Komparativ:Xist_SilberYist...  
1986         kaum1986    Die Konstruktion "Relativierung:kaumADJ" gehör...  
1987         sehr1987    Die Konstruktion "Intensivierung:sehrV" gehört...  

[562 rows x 3 columns]

##### Generieren neuer Sätze:

In [9]:
pseudowords = [
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_0_69.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_69_93.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_93_165.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_165_176.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_176_281.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_281_364.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_364_437.npy"),
    np.load(f"../../data/pseudowords/mbart/pseudowords_comapp_437_562.npy")
]
pseudowords = np.concatenate(pseudowords)
pseudowords

array([[ 0.01952364,  0.00048042,  0.05060846, ...,  0.04258203,
         0.05651806,  0.02429085],
       [-0.00994319, -0.00676309, -0.00601289, ..., -0.03203335,
        -0.03284921,  0.00028247],
       [-0.03243406, -0.00804963, -0.02258722, ...,  0.04839412,
         0.0025557 ,  0.01301195],
       ...,
       [ 0.00162825,  0.01625418,  0.0080498 , ..., -0.01727665,
         0.01961022, -0.00274661],
       [ 0.00529211,  0.00459571,  0.01711997, ...,  0.01059076,
        -0.01628351, -0.0441331 ],
       [-0.03313114,  0.00591089, -0.00273072, ..., -0.03151482,
         0.01069049,  0.00970152]], dtype=float32)

In [10]:
csv_data = []
# TODO mBART-Order laden
for i in range(0, 8):
    csv_data.append(pd.read_csv(f"../../data/pseudowords/mbart/order_{i}.csv", sep=";", index_col=0, header=None, quotechar="|", names=["order", "label"]))
csv_data = pd.concat(csv_data)
csv_data

,label
order,
0,"""""Was13"
1,"""647"
3,(1597
4,(1600
5,(1602
...,...
556,»Raus1316
557,Ähnlich123
559,ähnlich123


In [11]:
mbart_tokens = [d[0] for d in csv_data.values]

mbart_tokens, len(mbart_tokens)

(['""Was13',
  '"647',
  '(1597',
  '(1600',
  '(1602',
  '(1637',
  '(1641',
  '(1643',
  '(379',
  '(579',
  '(581',
  '(584',
  '(590',
  '(592',
  '(600',
  '(886',
  '(892',
  '(900',
  '(905',
  '(907',
  '(909',
  '(917',
  '(919',
  '(921',
  ')1597',
  ')1600',
  ')1637',
  ')1641',
  ')1643',
  ')1792',
  ')379',
  ')579',
  ')581',
  ')584',
  ')590',
  ')592',
  ')600',
  ')886',
  ')892',
  ')900',
  ')907',
  ')909',
  ')917',
  ')919',
  ')921',
  ')«579',
  ',1459',
  ',973',
  '-128',
  '-651',
  '-654',
  '-875',
  '-973',
  ':595',
  ':875',
  ':973',
  'Abstand683',
  'Allein20',
  'Aller1630',
  'Als1315',
  'Als133',
  'Als1770',
  'Am488',
  'Am492',
  'Am500',
  'Amerika605',
  'genauso122',
  'genauso98',
  'geschweige10',
  'gewesen1459',
  'gibt605',
  'gleich100',
  'gleich675',
  'gleich676',
  'gleich98',
  'gleiche104',
  'gleichen1777',
  'gleichkam676',
  'gleichkommen676',
  'gleichkommt676',
  'gleichkäme676',
  'gleicht132',
  'gold1554',
  'habe605'

Load the vanilla mbart model:

In [12]:
model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50", return_dict=True
) 
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50", src_lang="de_DE", tgt_lang="de_DE"
)
model.model.encoder.embed_tokens

Embedding(250054, 1024, padding_idx=1)

Add to existing embeddings:

In [13]:
combined_embeddings = torch.cat((model.model.encoder.embed_tokens.weight, torch.tensor(pseudowords)), dim=0)
model.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(combined_embeddings)
model.model.encoder.embed_tokens

Embedding(250560, 1024)

Add to existing tokens:

In [14]:
tokenizer.add_tokens(mbart_tokens)
model.resize_token_embeddings(len(tokenizer))

Embedding(250560, 1024)

In [15]:
model.to("cuda:0")

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250560, 1024)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250560, 1024)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05

Complete the cues:

In [16]:
def complete_cues(row):
    try:
        output_texts = []
        scores = []
        for cue, example in zip(row["cue"], row["example"]):
            assert row["pseudoword"].iloc[0] in mbart_tokens  # skip pseudoword embeddings that haven't been learned
            
            input_text = cue + " " + row["pseudoword"].iloc[0] + " <mask>"
            target_length = int(1.5 * len(example))  # allow double the length of the original sentence
            
            outputs = tokenizer(input_text, return_tensors="pt")
            outputs = model.generate(outputs["input_ids"].to("cuda:0"), max_length=target_length, num_return_sequences=1, num_beams=10, output_scores=True, return_dict_in_generate=True)
            output_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
            score = torch.exp(outputs.sequences_scores)
            output_texts += output_text
            
            scores.append(score)
        print(".", end="")
        return pd.Series({"construction": row["construction"], "pseudoword": row["pseudoword"].iloc[0], "orig_example": row["example"], "generated": str(output_texts), "scores": str([float(score) for score in scores])})
    except Exception as e:
        print(":", end="")
        return pd.Series({"construction": row["construction"], "pseudoword": row["pseudoword"].iloc[0], "orig_example": row["example"], "generated": str(e), "scores": "[-1.0]"})

examples_reset = examples.reset_index()
pseudoword_output_scores = examples_reset[["construction", "pseudoword", "example", "cue", "pseudoword"]].progress_apply(complete_cues, axis=1)
pseudoword_output_scores

  0%|          | 0/562 [00:00<?, ?it/s]

...:.........................................:....::..................................................................................................................:................................................................:........................:::::::::::::::::::::::::::::::::::::::.::::::::.....................................................................:.....................................................................:.................................:....................................................................................

,construction,pseudoword,orig_example,generated,scores
0,5,Und5,"[""""Nicht herauskaufen"""", """"nicht erpressen las...","['""""Nicht herauskaufen"""", """"nicht erpressen la...","[0.9136524200439453, 0.44515761733055115, 0.51..."
1,5,erst5,[Trainer Lucien Favre hatte schon seine beiden...,['Trainer Lucien Favre hatte schon seine beide...,"[0.7875338792800903, 0.7785300612449646, 0.742..."
2,5,gar5,[Es hat Afghanistan nicht stabilisiert und sch...,['Es hat Afghanistan nicht stabilisiert gar5 -...,"[0.5824133157730103, 0.6445226073265076, 0.719..."
3,5,nicht5,[Es hat Afghanistan nicht stabilisiert und sch...,,[-1.0]
4,5,recht5,[Trainer Lucien Favre hatte schon seine beiden...,['Trainer Lucien Favre hatte schon seine beide...,"[0.7853999733924866, 0.8219645023345947, 0.791..."
...,...,...,...,...,...
557,1884,Gold1884,"[Schweigen ist Silber , reden ist Gold ., Schw...","['Schweigen ist Silber, reden ist ', 'Schweige...","[0.7522048950195312, 0.8172914981842041, 0.784..."
558,1884,Silber1884,"[Schweigen ist Silber , reden ist Gold ., Schw...","['Schweigen ist ', 'Schweigen ist ', 'Ange...","[0.7068747878074646, 0.7068747878074646, 0.643..."
559,1884,ist1884,"[Schweigen ist Silber , reden ist Gold ., Schw...","['Schweigen ist1884 (Englisch)', 'Schweigen is...","[0.4845588505268097, 0.4845588505268097, 0.498..."
560,1986,kaum1986,[Die Vorhut vor 20.000 Jahren war für das Ries...,"['Die Vorhut vor 20.000 Jahren ', 'Vorhut vo...","[0.6681877374649048, 0.5497788190841675, 0.670..."


In [17]:
examples = pseudoword_output_scores[["pseudoword", "generated", "scores"]]

examples

,pseudoword,generated,scores
0,Und5,"['""""Nicht herauskaufen"""", """"nicht erpressen la...","[0.9136524200439453, 0.44515761733055115, 0.51..."
1,erst5,['Trainer Lucien Favre hatte schon seine beide...,"[0.7875338792800903, 0.7785300612449646, 0.742..."
2,gar5,['Es hat Afghanistan nicht stabilisiert gar5 -...,"[0.5824133157730103, 0.6445226073265076, 0.719..."
3,nicht5,,[-1.0]
4,recht5,['Trainer Lucien Favre hatte schon seine beide...,"[0.7853999733924866, 0.8219645023345947, 0.791..."
...,...,...,...
557,Gold1884,"['Schweigen ist Silber, reden ist ', 'Schweige...","[0.7522048950195312, 0.8172914981842041, 0.784..."
558,Silber1884,"['Schweigen ist ', 'Schweigen ist ', 'Ange...","[0.7068747878074646, 0.7068747878074646, 0.643..."
559,ist1884,"['Schweigen ist1884 (Englisch)', 'Schweigen is...","[0.4845588505268097, 0.4845588505268097, 0.498..."
560,kaum1986,"['Die Vorhut vor 20.000 Jahren ', 'Vorhut vo...","[0.6681877374649048, 0.5497788190841675, 0.670..."


#### Anzahl neuer Sätze

In [18]:
examples.to_csv(f"../../out/comapp/mbart/data_mbart.tsv", sep="\t", decimal=",")
examples.to_excel(f"../../out/comapp/mbart/data_mbart.xlsx")

In [19]:
pseudoword_output_scores.to_csv("../../out/comapp/mbart/data_mbart_complete.tsv", sep="\t", decimal=",")